### **0. Import libraries**

In [ ]:
import pandas as pd
import time
import requests
import json
from os import path
import os
import re
import numpy as np

### **1. Functions**

In [ ]:
# avoid an error if value empty
def safe_get(dic, *keys):
    for key in keys:
        if dic is None:
            return None
        dic = dic.get(key)
    return dic

# extract the food as a list
def process_food(food_list):
    if food_list:
        return ", ".join([food.get('name', 'Unknown') for food in food_list])
    return None

# extract the flavor as a list
def process_flavor(flavor_list):
    group_names = []
    if flavor_list:
        for flavor in flavor_list:
            group_names.append(flavor.get('group', 'Unknown'))
    return ', '.join(group_names)

# extract the flavor as a list
def process_grapes(grape_list):
    if grape_list:
        return ", ".join([grape.get('name', 'Unknown') for grape in grape_list])
    return None

In [ ]:
# to catch the information of the specific wine 
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = f"https://www.vivino.com/api/wines/{wine_id}/reviews?per_page=50&year={year}&page={page}" 
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  
        data = response.json()
    # Avoid break-up HTTP Error
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
        return None
    # Avoid break-up Connection Error
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
        return None
    # Avoid break-up Time-ban error
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
        return None
    # Avoid break-up Time-ban error
    except requests.exceptions.RequestException as err:
        print("Something went wrong:", err)
        return None
    # Avoid break-up Time-ban error
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", str(e))
        return None

    # Check if the response data is empty
    if not data:
        print("Empty response from the API")
        return None

    return data

### **2. features df**

In [ ]:
# Maximum duration in seconds to avoid too huge datasets
max_duration_seconds = 20000
start_time = time.time()

# Start time
start_time = time.time()

# Fetch data from all pages
all_results = []
num_pages_to_fetch = 1

for page_number in range(1, num_pages_to_fetch + 1):
    print(f'Fetching data for page {page_number}')
    
    r = requests.get(
        "https://www.vivino.com/api/explore/explore",
        params={
            "country_code": ["de"], # base of the price comparison
            "country_codes[]": ["pt"], # country of production
            "currency_code": "EUR", # price unit
            "grape_filter": "varietal", # different grapes
            "min_rating": "1", # minium 
            "order_by": "price", # for reproducability
            "order": "asc", # low to high
            "page": page_number,
            "price_range_max": "500", # upper level price range
            "price_range_min": "0" # lower level price range 
        },
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
        },
    )

    results = [
        (
            safe_get(t, "vintage", "wine", "id"), # wine_id
            safe_get(t, "vintage", "seo_name"), # website wine name  
            safe_get(t, "vintage", "id"), # vintage id
            safe_get(t, "vintage", "name"),  # wine name
            safe_get(t, "vintage", "wine", "winery", "name"),  # winery name
            safe_get(t, "vintage", "year"),  # year of production
            safe_get(t, 'vintage', 'wine', 'region', 'country', 'name'),  # country of region
            safe_get(t, 'vintage', 'wine', 'region', "name"),  # region name
            safe_get(t, "vintage","wine","style", "name"), # wine style
            safe_get(t, "vintage","wine","style", "description"), # wine style description
            safe_get(t, "vintage", "statistics", "ratings_average"), # ratings average
            safe_get(t, "vintage", "statistics", "ratings_count"),  # number of ratings
            safe_get(t, "vintage","statistics","wine_ratings_average"), # accumulated ratings average for all vintages
            safe_get(t, "vintage","statistics","wine_ratings_count"), # accumulated number of ratings 
            safe_get(t, "price", "amount"), # available price in Germany
            safe_get(t, "price", "discounted_from"), # discount at the moment
            safe_get(t, "price", "type"), # different price types
            process_grapes(safe_get(t, "vintage", "wine", "style", "grapes")), # grapes of the wine
            safe_get(t, "vintage", "wine", "style", "interesting_facts"), # interesting facts about the wine
            safe_get(t, "vintage", "wine", "style", "body"), # body rating of the wine
            safe_get(t, "vintage", "wine", "style", "body_description"), # body description of the wine
            safe_get(t, "vintage", "wine", "style", "wine_type_id"), # wine type
            safe_get(t, "vintage", "wine", "style", "varietal_name"), # wine type vivino
            safe_get(t, "price", "bottle_type", "name"), # bottle type
            safe_get(t, "vintage", "wine","taste", "structure", "acidity"),  # acidity
            safe_get(t, "vintage", "wine","taste", "structure", "sweetness"),  # sweetness
            safe_get(t, "vintage", "wine","taste", "structure", "intensity"), # intensity
            safe_get(t, "vintage", "wine","taste", "structure", "tannin"), # tanin
            safe_get(t, "vintage", "wine","taste", "structure", 'fizziness'),  # fizziness
            process_food(safe_get(t, "vintage", "wine", "style", "food")), # food pairing
            process_flavor(safe_get(t, "vintage", "wine", "taste", "flavor")), # aroma word
        )
        for t in r.json()["explore_vintage"]["matches"] # to get access to the data 
    ]
    
    all_results.extend(results) # create a list 



# Saving the results in a dataframe
feature_df = pd.DataFrame(
    all_results,
    columns=["wine_id","seo-name", "vintage_id",
             "wine","winery", "year", "country", "region", "wine_style", "wine_style_description",
             "ratings_average", "ratings_number", "all_vintage_ratings_average","all_vintage_ratings_count", 
             "price", "original_price", "price_type",
             "grapes", "interesting_facts", 
             "body_rating", "body_description",'vivino_type_id', 'genre', "bottle_type", 
             "acidity", "sweetness","intensity", "tannin",'fizziness', "suggested_food_pairining", "flavor_words"
    ]
) # create the dataframe of features

print(feature_df.describe())

feature_df.to_csv('20231006_pt_feature.csv')

### **3. review_df**

In [ ]:
# Maximum duration in seconds to avoid too huge datasets
max_duration_seconds = 20000
start_time = time.time()

# Start time
start_time = time.time()

# initalize the list for the specific rating 
ratings = [] 

for _, row in feature_df.iterrows():
    page = 1
    elapsed_time = time.time() - start_time

    if elapsed_time > max_duration_seconds:
        print("Maximum duration reached. Stopping the data collection.")
        break

    while page < 100:
        print(
            f'Getting info about wine {row["wine_id"]}-{row["year"]} Page {page}'
        )

        d = get_wine_data(row["wine_id"], row["year"], page)

        if d is None or not d.get("reviews"):
            break
# to have just english comments for topic 
        for r in d["reviews"]:
            if r["language"] != "en":
                continue 

        for r in d["reviews"]:
            if r["language"] != "en":
                continue 

            ratings.append(
                    [
                        row["wine_id"], # orignal year
                        row["year"], # wine id
                        safe_get(r, "id"), # review_id
                        safe_get(r, "user", "id"),  # user id 
                        safe_get(r, "user", "alias"),  # user name
                        safe_get(r, "rating"),  # given rating
                        safe_get(r, "note"),  # given note
                        safe_get(r, "created_at"),  # time stamp
                        safe_get(r, "vintage", "year"),  # vintage referred vintage
                        safe_get(r, "activity", "statistics", "likes_count"),  # likes received
                        safe_get(r, "activity", "statistics", "comments_count"),  # comments received
                        safe_get(r, "user", "language"),  # user languag
                        safe_get(r, "user", "statistics", "followers_count"),  # follower
                        safe_get(r, "user", "statistics", "followings_count"),  # following
                        safe_get(r, "user", "statistics", "ratings_count"),  # all ratings
                        safe_get(r, "user", "statistics", "ratings_sum"),  # total rating sum
                        safe_get(r, "user", "statistics", "reviews_count"),  # total reviews given
                        safe_get(r, "user", "statistics", "purchase_order_count") # purchases on vivino
                            ]
                )

        page += 1

ratings_df = pd.DataFrame(
    ratings, columns=["wine_id", "year",  "review_id", "user_id","user_name","review_rating", "review", "createdat", "review_referring_year","likes_on_review", "comments_on_review", "user_language", 
                      "user_follower", "user_following", "user_total_given_ratings", "user_total_sum_ratings","user_total_given_reviews","user_purchase_made"
                      ]
)

ratings_df.to_csv('20231006_pt_reviews.csv')
print("done")

#### 4. alcohol_df

In [ ]:
# working directory
data_directory = '/Users/lukas/OneDrive/Business Analytics/5 Trimester/Conduct your own end-to-end analytics project/Code/Upload'

# features
df_feature = pd.read_csv(path.join(data_directory, '20231006_pt_feature.csv'),verbose=False)

# filter for interest 
wine = df_feature[["seo-name", "wine_id"]]

# delete the last five rows as it the pattern of the structure 
wine["cleaned_seo_name"] = wine["seo-name"].str[:-5]

# to ext
seo_list = []
id_list = []
alcohol_list = []


def get_alcohol(seo, wine):
    url = url = f'https://www.vivino.com/DE/en/{seo}/w/{wine_id}'
    r = requests.get(url,
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    })
    # this gets the javascript object
    res = re.search(r"^.*window\.__PRELOADED_STATE__\.winePageInformation\s*=\s*(.*});", r.text, re.MULTILINE)

    # Check if a match was found
    if res:
        data = json.loads(res.group(1))
        # Continue with the rest of your code
        alcohol = data["vintage"]["wine"]["alcohol"]
    else:
        # Handle the case when no match was found
        print("No match found in the response text.")
        # You might want to add some error handling or 
        alcohol = np.nan


    return alcohol

counter = 0

for index, row in wine.iterrows():
    wine_id = row['wine_id']
    cleaned_seo_name = row['cleaned_seo_name']
    alcohol = get_alcohol(wine_id,  cleaned_seo_name)

    seo_list.append(cleaned_seo_name)
    id_list.append(wine_id)
    alcohol_list.append(alcohol)

    counter += 1
    print(counter)



df_alcohol = pd.DataFrame({'wine_id' : id_list,
                                'seo_cleaned' : seo_list,
                                'alcohol' : alcohol_list }, 
                                columns=['wine_id','seo_cleaned', 'alcohol'])